In [1]:
!pip install --q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.7/160.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.5 MB/s eta

In [2]:
llm=ChatGroq(temperature=0,
             model_name="llama3-70b-8192",
             api_key='gsk_ZjN*********')

In [3]:
planner = Agent(
    llm=llm,
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True
)

writer = Agent(
    llm=llm,
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working wqon a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

editor = Agent(
    llm=llm,
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

In [4]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
  "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [5]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [6]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I've gathered all the necessary information and I'm ready to create a comprehensive content plan for the blog article on Artificial Intelligence.

Final Answer:

**Content Plan: Artificial Intelligence**

**Target Audience:**

* Demographics: Business owners, entrepreneurs, and professionals in the tech industry
* Interests: Staying up-to-date with the latest technological advancements, improving business efficiency, and exploring innovative solutions
* Pain Points: Understanding the complexities of AI, implementing AI solutions, and staying ahe

In [7]:
from IPython.display import Markdown
Markdown(result)

**The AI Revolution: Transforming Industries and Revolutionizing the Way We Work and Live**
====================================================================

**Introduction**
---------------

Artificial Intelligence (AI) has been a buzzword in the tech industry for decades, but its impact is now being felt across various sectors. From chatbots and virtual assistants to image recognition and natural language processing, AI is transforming the way we work and live. As businesses strive to stay ahead of the competition, understanding AI and its applications is crucial for success. In this article, we'll delve into the world of AI, exploring its history, types, and latest trends, as well as its benefits and challenges for businesses.

AI is no longer just a futuristic concept; it's a reality that's changing the way we live and work. With its ability to learn, problem-solve, and make decisions, AI is revolutionizing industries and transforming the way we interact with technology.

**Understanding Artificial Intelligence**
-------------------------------------

AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. The history of AI dates back to the 1950s, but it wasn't until the 21st century that AI started to gain traction. There are three types of AI: Narrow or Weak AI, General or Strong AI, and Superintelligence. Narrow AI is designed to perform a specific task, such as facial recognition or language translation. General AI, on the other hand, is capable of performing any intellectual task that a human can. Superintelligence refers to AI that surpasses human intelligence.

AI applications are numerous, including Machine Learning, Deep Learning, and Natural Language Processing. Machine Learning enables systems to learn from data and improve their performance over time. Deep Learning is a subset of Machine Learning that uses neural networks to analyze data. Natural Language Processing allows systems to understand and generate human-like language.

**Latest Trends and Advancements in AI**
--------------------------------------

### Natural Language Processing (NLP)

NLP has revolutionized the way we interact with machines. Chatbots and virtual assistants, such as Amazon's Alexa and Google's Duplex, can understand and respond to voice commands. Language translation has also become more accurate, enabling businesses to communicate with customers worldwide. With NLP, businesses can analyze customer feedback, sentiment, and preferences, enabling them to make data-driven decisions.

### Computer Vision

Computer Vision has enabled machines to recognize and interpret visual data. Image recognition, object detection, and facial recognition are just a few examples of Computer Vision applications. Self-driving cars, surveillance systems, and medical diagnosis are some of the industries that benefit from Computer Vision. With Computer Vision, businesses can analyze and interpret visual data, enabling them to make informed decisions.

### Edge AI

Edge AI refers to the decentralized processing of AI data in real-time. This enables devices to make decisions without relying on cloud-based servers, reducing latency and improving performance. Edge AI has numerous applications, including smart homes, autonomous vehicles, and industrial automation. With Edge AI, businesses can process data in real-time, enabling them to respond quickly to changing circumstances.

### Explainable AI (XAI)

XAI is a growing concern in the AI community. As AI systems become more complex, it's essential to understand how they make decisions. XAI enables businesses to interpret and explain AI-driven decisions, ensuring transparency and accountability. With XAI, businesses can build trust with their customers, ensuring that AI systems are fair and unbiased.

**Key Players and Innovations in AI**
--------------------------------------

### Google

Google has been at the forefront of AI innovation, with projects like DeepMind, TensorFlow, and Duplex. DeepMind is a leading AI research organization, while TensorFlow is an open-source Machine Learning framework. Duplex, a chatbot that can make human-like phone calls, has the potential to revolutionize customer service. Google's innovations have enabled businesses to build and deploy AI models quickly and efficiently.

### Microsoft

Microsoft has made significant contributions to AI, with its Azure Machine Learning platform and DeepFace project. Azure Machine Learning enables businesses to build, deploy, and manage AI models, while DeepFace is a facial recognition system that can identify individuals with high accuracy. Microsoft's innovations have enabled businesses to integrate AI with existing systems, improving efficiency and productivity.

### Amazon

Amazon has invested heavily in AI, with its SageMaker and Alexa projects. SageMaker is a cloud-based Machine Learning platform, while Alexa is a virtual assistant that can understand and respond to voice commands. Amazon's innovations have enabled businesses to build and deploy AI models quickly and efficiently, improving customer experience and loyalty.

**Implementing AI in Business**
---------------------------------

### Benefits of AI Adoption

AI can bring numerous benefits to businesses, including increased efficiency, improved customer experience, and competitive advantage. AI can automate repetitive tasks, freeing up employees to focus on higher-value tasks. AI-powered chatbots can provide 24/7 customer support, improving customer satisfaction and loyalty. With AI, businesses can analyze large datasets, enabling them to make data-driven decisions.

### Challenges of AI Implementation

However, AI implementation comes with its own set of challenges. Data quality, integration, and ethics are just a few concerns that businesses need to address. AI systems require high-quality data to learn and improve, and integrating AI with existing systems can be complex. Ethics is another crucial aspect, as AI systems can perpetuate biases and discrimination.

### Best Practices for AI Adoption

To overcome these challenges, businesses should start small, focusing on specific AI applications that can bring immediate value. It's essential to invest in employee training, ensuring that employees understand AI and its applications. Businesses should also prioritize ethics, ensuring that AI systems are transparent, accountable, and fair. By following best practices, businesses can ensure responsible AI development and maximize the benefits of AI adoption.

**Conclusion**
----------

AI is transforming industries and revolutionizing the way we work and live. From NLP and Computer Vision to Edge AI and XAI, the latest trends and advancements in AI are numerous. Key players like Google, Microsoft, and Amazon are driving innovation, and businesses are starting to reap the benefits of AI adoption. However, challenges remain, and it's essential to address these concerns to ensure responsible AI development. As we move forward, it's crucial to stay informed and adapt to the changing AI landscape. Start exploring AI solutions for your business today and discover the transformative power of AI.